In [1]:
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
import random
from sklearn.preprocessing import OneHotEncoder

# Load the new datasets
#df_movies_all = pd.read_csv("data_analysis/movies.txt", sep=',')
#df_cast_people = pd.read_csv("data_analysis/cast.txt", sep=',')
#df_movie_companies = pd.read_csv("data_analysis/companies.txt", sep=',')
#df_ratings = pd.read_csv("data_analysis/ratings.txt", sep=',')
#df_users = pd.read_csv("data_analysis/users.txt", sep=',')
#movies_cast_company = pd.read_csv("data_analysis/movies_cast_company.txt", sep=',')

final_dataset = pd.read_csv('data_analysis/final_dataset.csv')


# Filter users with more than 200 interactions
user_interactions = final_dataset.groupby('user_id').size()
user_indexes = user_interactions[user_interactions > 200].index
filtered_ratings = final_dataset[final_dataset.user_id.isin(user_indexes)]

# Ensure movie filtering matches the filtered ratings
filtered_movies = final_dataset[final_dataset['movie_id_ml'].isin(filtered_ratings['movie_id_ml'])]

# Drop NaN values and unused columns for the movie dataset
filtered_movies = filtered_movies.dropna(axis=1)


print(filtered_movies)



       user_id  movie_id_ml  rating     rating_timestamp  \
0          186          302       3  1998-04-04 19:22:22   
1          186         1042       5  1997-11-08 21:13:52   
2          186           98       5  1998-04-04 19:57:39   
3          186          327       3  1998-04-04 19:23:26   
4          186          332       4  1998-04-04 19:56:15   
...        ...          ...     ...                  ...   
79784      358         1149       3  1998-03-30 15:00:43   
79785      358          584       4  1998-03-30 14:58:33   
79786      358          918       1  1998-04-16 12:54:14   
79787      358          896       4  1998-03-30 14:44:37   
79788      358         1159       5  1998-03-30 14:53:37   

                          title  release  \
0             l.a. confidential     1997   
1                    just cause     1995   
2      the silence of the lambs     1991   
3                      cop land     1997   
4                kiss the girls     1997   
...            

In [2]:
from sklearn.model_selection import train_test_split
import random
import pandas as pd

# Select a random user for testing
test_user = random.choice(filtered_ratings.user_id.unique())

# Function to prepare training and testing data for a specific user
def get_data_for_user(filtered_ratings, filtered_movies, user_id, sample_size=2000):
    """
    Prepare training and testing data for a specific user, with optional data sampling.
    """
    # Filter ratings for the selected user
    user_ratings = filtered_ratings[filtered_ratings['user_id'] == user_id]
    print(f"User Ratings Columns: {user_ratings.columns}")  # Debug: Check columns in user_ratings

    # Filter movies rated by the selected user
    user_movies = filtered_movies[filtered_movies['movie_id_ml'].isin(user_ratings['movie_id_ml'])]
    print(f"User Movies Columns: {user_movies.columns}")  # Debug: Check columns in user_movies

    # Merge user ratings with movie data
    user_data = user_movies.merge(
        user_ratings,
        on='movie_id_ml',  # Merge based on the movie ID
        suffixes=('_movie', '_rating')
    )
    print(f"Merged User Data Columns: {user_data.columns}")  # Debug: Check columns after merging

    # Drop unnecessary columns
    if 'user_id_movie' in user_data.columns:
        user_data = user_data.drop(['user_id_movie'], axis=1)
    if 'rating_timestamp_movie' in user_data.columns:
        user_data = user_data.drop(['rating_timestamp_movie'], axis=1)

    # Ensure the correct column name for 'rating' is used
    if 'rating_rating' in user_data.columns:
        rating_column = 'rating_rating'
    else:
        raise KeyError("'rating' column not found in merged user_data.")

    # Separate features (X) and target (y)
    X = user_data.drop(rating_column, axis=1)
    y = user_data[rating_column]

    # Handle non-numeric columns using one-hot encoding
    X = pd.get_dummies(X, drop_first=True)

    # Randomly sample the data, limiting it to the specified sample size
    sample_indices = random.sample(range(X.shape[0]), min(sample_size, X.shape[0]))
    X_sampled = X.iloc[sample_indices]
    y_sampled = y.iloc[sample_indices]

    # Split the sampled data into training and testing sets
    return train_test_split(X_sampled, y_sampled, test_size=0.3, random_state=42)


# Generate training and testing datasets for the selected user
try:
    X_train, X_test, y_train, y_test = get_data_for_user(filtered_ratings, filtered_movies, test_user)

    # Output shapes of the resulting datasets
    print(f"X_train shape: {X_train.shape}")
    print(f"y_train shape: {y_train.shape}")
    print(f"X_test shape: {X_test.shape}")
    print(f"y_test shape: {y_test.shape}")
except KeyError as e:
    print(f"KeyError: {e}")


User Ratings Columns: Index(['user_id', 'movie_id_ml', 'rating', 'rating_timestamp', 'title',
       'release', 'url', 'unknown', 'action', 'adventure', 'animation',
       'childrens', 'comedy', 'crime', 'documentary', 'drama', 'fantasy',
       'noir', 'horror', 'musical', 'mystery', 'romance', 'scifi', 'thriller',
       'war', 'western', 'movie_id', 'keyword', 'cast', 'company', 'user_age',
       'user_gender', 'user_occupation', 'user_zipcode', 'tconst', 'nconst',
       'director'],
      dtype='object')
User Movies Columns: Index(['user_id', 'movie_id_ml', 'rating', 'rating_timestamp', 'title',
       'release', 'url', 'unknown', 'action', 'adventure', 'animation',
       'childrens', 'comedy', 'crime', 'documentary', 'drama', 'fantasy',
       'noir', 'horror', 'musical', 'mystery', 'romance', 'scifi', 'thriller',
       'war', 'western', 'movie_id', 'keyword', 'cast', 'company', 'user_age',
       'user_gender', 'user_occupation', 'user_zipcode', 'tconst', 'nconst',
       'd

In [54]:
print(f"Original X shape: {X_train.shape}, y shape: {y_train.shape}")
print(f"After splitting: X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"After adjustments: X_train shape: {X_train.shape}, y_train_adjusted shape: {y_train_adjusted.shape}")


Original X shape: (30183, 4553), y shape: (30183,)
After splitting: X_train shape: (30183, 4553), y_train shape: (30183,)
After adjustments: X_train shape: (30183, 4553), y_train_adjusted shape: (298,)


In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from surprise import SVD, NMF, Dataset, Reader
from surprise.model_selection import GridSearchCV as SurpriseGridSearch, cross_validate
from xgboost import XGBClassifier
from joblib import Parallel, delayed
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

#X_train, X_test, y_train, y_test = get_data_for_user(filtered_ratings, filtered_movies, test_user)
reader = Reader(rating_scale=(1, 5))  # Adjust the rating scale if needed

# Convert the DataFrame into a Surprise dataset
data = Dataset.load_from_df(filtered_ratings[['user_id', 'movie_id', 'rating']], reader)

param_grid = {
    'LogisticRegression': {
        'model': LogisticRegression(),
        'params': {
            'C': [0.1, 1, 10],
            'solver': ['liblinear', 'lbfgs'],
            'max_iter': [1000]
        }
    },
    'RandomForest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [10, 50, 100],
            'max_depth': [None, 10, 20]
        }
    },
    'DecisionTree': {
        'model': DecisionTreeClassifier(),
        'params': {
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5, 10]
        }
    },
    'MLPClassifier': {
        'model': MLPClassifier(max_iter=1000, random_state=42),
        'params': {
            'hidden_layer_sizes': [(100,), (100, 50), (150,), (150, 75)],
            'activation': ['relu'],
            'solver': ['adam'],
            'alpha': [0.0001, 0.001, 0.01]
        }
    },
    'SVD': {
        'model': SVD,
        'params': {
            'n_factors': [50, 100, 150],
            'n_epochs': [20, 50],
            'lr_all': [0.005, 0.01]
        }
    },
    'NMF': {
        'model': NMF,
        'params': {
            'n_factors': [10, 20],
            'n_epochs': [20, 50]
        }
    },
    'XGBoost': {
        'model': XGBClassifier(),
        'params': {
            'n_estimators': [50, 100],
            'learning_rate': [0.1, 0.01],
            'max_depth': [3, 5, 10]
        }
    }
}

# Perform grid search for Scikit-learn and Surprise models
best_models = {}
for model_name, model_info in param_grid.items():
    model = model_info['model']
    params = model_info['params']
    print(f"Evaluating {model_name}...")
    
    if model_name in ['SVD', 'NMF']:
        # Use Surprise's GridSearchCV for hyperparameter tuning
        gs = SurpriseGridSearch(param_grid=params, measures=['rmse'], cv=5, algo_class=SVD)
        gs.fit(data)
        best_params = gs.best_params['rmse']
        best_score = gs.best_score['rmse']
        best_models[model_name] = (model(**best_params), best_score)
        print(f"Best RMSE for {model_name}: {best_score}, Params: {best_params}")
    
    else:
        # Scale the data before fitting the models that require scaling (e.g., Logistic Regression)
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_train_scaled_subset = X_train_scaled[:1000]  # Use the first 1000 samples as a subset
        y_train_subset = y_train[:1000]
        label_encoder = LabelEncoder()
        y_train_subset_encoded = label_encoder.fit_transform(y_train_subset)

        # Perform Grid Search for Scikit-learn compatible models
        grid_search = GridSearchCV(estimator=model, param_grid=params, cv=5, scoring='accuracy', n_jobs=2)
        grid_search.fit(X_train_scaled_subset, y_train_subset_encoded)
        best_models[model_name] = (grid_search.best_estimator_, grid_search.best_score_)
        print(f"Best model for {model_name}: {grid_search.best_estimator_}, Score: {grid_search.best_score_}")

# Display results
print("\nBest Models and Scores:")
for model_name, model_info in best_models.items():
    if model_name in ['SVD', 'NMF']:
        # For SVD and NMF, display the best RMSE and the best hyperparameters directly
        print(f"{model_name}: Best RMSE = {model_info[1]}, Params = {model_info[0]}")
    else:
        # For other models (like RandomForest, MLPClassifier, etc.), display accuracy and best estimator
        print(f"{model_name}: Best Accuracy = {model_info[1]}, Best Estimator = {model_info[0]}")


Evaluating LogisticRegression...
Best model for LogisticRegression: LogisticRegression(C=0.1, max_iter=1000, solver='liblinear'), Score: 0.962
Evaluating RandomForest...
Best model for RandomForest: RandomForestClassifier(), Score: 0.967
Evaluating DecisionTree...
Best model for DecisionTree: DecisionTreeClassifier(), Score: 0.96
Evaluating MLPClassifier...


c:\Users\Dell\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:412: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.array(param_list)


Best model for MLPClassifier: MLPClassifier(hidden_layer_sizes=(150,), max_iter=1000, random_state=42), Score: 0.9710000000000001
Evaluating SVD...
Best RMSE for SVD: 0.8981705712568898, Params: {'n_factors': 150, 'n_epochs': 50, 'lr_all': 0.01}
Evaluating NMF...
Best RMSE for NMF: 0.9194070624988008, Params: {'n_factors': 20, 'n_epochs': 20}
Evaluating XGBoost...
Best model for XGBoost: XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_c

In [3]:
print(movies_cast_company.columns)
print(df_ratings.columns)

Index(['movie_id_ml', 'title', 'release', 'url', 'unknown', 'action',
       'adventure', 'animation', 'childrens', 'comedy', 'crime', 'documentary',
       'drama', 'fantasy', 'noir', 'horror', 'musical', 'mystery', 'romance',
       'scifi', 'thriller', 'war', 'western', 'movie_id', 'keyword', 'cast',
       'company'],
      dtype='object')
Index(['user_id', 'movie_id_ml', 'rating', 'rating_timestamp'], dtype='object')
